In [20]:
import scanpy as sc
import numpy as np
import pandas as pd

In [21]:
dir_path = "/home/krushna/Documents/scDREAMER/Data_integration/SCRNA_Datasets/All_h5ad/"

batch_key_dic = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'Batch'
                }
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Pancreas' : 'Group'
                    }


In [22]:
dataset = "Lung"
adata =sc.read_h5ad(dir_path+dataset+'.h5ad')

In [23]:
batch = batch_key_dic[dataset]
cell_type = cell_type_key_dic[dataset]

In [24]:
no_of_batch = adata.obs[batch].value_counts()
pd.DataFrame(no_of_batch).sort_values('batch',ascending=False)[:3]

,batch
4,3763
B2,3297
6,3229


In [25]:
meta_data = adata.obs[[batch,cell_type]]

In [28]:
len(np.unique(meta_data[batch]))

16

In [26]:

for batch_value in np.unique(meta_data[batch]):
    meta_data_batch= meta_data[meta_data[batch]==batch_value]
    print(batch_value)
    print(meta_data_batch[cell_type].value_counts().sort_values(ascending=False)[:3])
    

1
Neutrophils_IL1R2    438
Endothelium          157
T/NK cell            142
Name: cell_type, dtype: int64
2
Neutrophil_CD14_high    1227
Dendritic cell           230
T/NK cell                209
Name: cell_type, dtype: int64
3
Type 2            135
Dendritic cell     98
Mast cell          98
Name: cell_type, dtype: int64
4
Macrophage    1828
Type 2        1419
Fibroblast     212
Name: cell_type, dtype: int64
5
Macrophage    1105
Type 2         356
T/NK cell      190
Name: cell_type, dtype: int64
6
Macrophage        1533
Type 2             822
Dendritic cell     495
Name: cell_type, dtype: int64
A1
Basal 1      530
Ciliated     488
Secretory    388
Name: cell_type, dtype: int64
A2
Basal 1     485
Basal 2     353
Ciliated    346
Name: cell_type, dtype: int64
A3
Ciliated     487
Basal 1      260
Secretory    232
Name: cell_type, dtype: int64
A4
Basal 1     218
Ciliated    195
Basal 2     157
Name: cell_type, dtype: int64
A5
Basal 2      1465
Ciliated      442
Secretory     215
Name: cell

In [10]:
#immune_human
to_remove = pd.DataFrame(
[    
    ['10X', 'CD14+ Monocytes', 3388],
    ['Freytag','CD4+ T cells', 1238],
    ['Oetjen_A','CD4+ T cells', 577],
    ['Oetjen_P','Erythrocytes',1219],
    ['Oetjen_U', 'CD20+ B cells', 417],
    ['Sun_sample1_CS','CD4+ T cells', 1054],
    ['Sun_sample2_KC', 'CD14+ Monocytes', 522],
    ['Sun_sample3_TB', 'CD14+ Monocytes', 372],
    ['Sun_sample4_TC', 'NK cells' ,464],
    ['Villani','Monocyte-derived dendritic cells',534]
],
    columns = [batch,cell_type,'count']
)

In [29]:
#Lung
to_remove = pd.DataFrame(
[    
    ['1','Neutrophils_IL1R2',438],
    ['2', 'Neutrophil_CD14_high', 1227],
    ['3', 'Type 2', 135],
    ['4','Macrophage', 1828],
    ['5', 'Macrophage', 1105],
    ['6', 'Type 2', 822],
    ['A1', 'Basal 1', 530],
    ['A2', 'Basal 1' ,485],
    ['A3', 'Ciliated', 487],
    ['A4', 'Ciliated',195],
    ['A5', 'Basal 2', 1465],
    ['A6', 'Ciliated', 622],
    ['B1', 'Type 2', 749],
    ['B2', 'Macrophage', 1293],
    ['B3', 'T/NK cell',189],
    ['B4', 'B cell', 1211]
],
    columns = [batch,cell_type,'count']
)


In [30]:
to_remove

,batch,cell_type,count
0,1,Neutrophils_IL1R2,438
1,2,Neutrophil_CD14_high,1227
2,3,Type 2,135
3,4,Macrophage,1828
4,5,Macrophage,1105
5,6,Type 2,822
6,A1,Basal 1,530
7,A2,Basal 1,485
8,A3,Ciliated,487
9,A4,Ciliated,195


In [31]:
sum(to_remove['count'])

12781

In [32]:
len(adata.obs['batch'])

32472

In [33]:
cell_to_remove = np.array([False]* len(adata.obs['batch']),dtype='bool')
np.unique(cell_to_remove,return_counts=True)

(array([False]), array([32472]))

In [34]:
for batch_val,cell_type_val in zip(to_remove[batch],to_remove[cell_type]):
    cell_to_remove = cell_to_remove | (np.array(meta_data[batch]==batch_val)
                                       & np.array(meta_data[cell_type] == cell_type_val))

In [35]:
np.unique(cell_to_remove,return_counts=True)

(array([False,  True]), array([19691, 12781]))

In [36]:
adata_sub = adata[~cell_to_remove]

In [37]:
adata_sub

View of AnnData object with n_obs × n_vars = 19691 × 15148
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    layers: 'counts'

In [38]:
adata_sub.write(dir_path+dataset+'_removed_top_cells.h5ad')